In [4]:
import pandas as pd
import numpy as np

data_1 = pd.read_csv('bina_az_19092023.csv')
data_2 = pd.read_csv('bina_az_21092023.csv')
frames = pd.concat([data_1, data_2], copy=True,   ignore_index=True,).drop_duplicates(keep='last').dropna(ignore_index=True)
frames['view_count']=frames['view'].str.extract(r'(?i)(?:Baxışların sayı|Просмотров): (\d+)')
seller_type_mapping = {
    'vasitəçi (agent)': 0,
    'посредник (агент)': 0,
    'собственник': 1,
    'mülkiyyətçi': 1
}

frames['seller_type_encoded'] = frames['seller_type'].replace(seller_type_mapping)
frames.drop('seller_type', axis=1, inplace=True)
frames['price']=frames['price'].str.replace(' ', '').astype(float)
frames=frames.sort_values(by='price', ascending=True)
building_type_mapping = {
    'Köhnə tikili': 'Old building',
    'Yeni tikili': 'New building',
    'Ofis': 'Office',
    'Офис': 'Office',
    'Объект': 'Object',
    'Дом / Дача': 'House / Cottage',
    'Вторичка': 'Secondary',
    'Həyət evi / Bağ evi': 'House / Garden house',
    'Новостройка': 'New building',
    'Obyekt': 'Object',
    'Mənzil': 'Apartment',
    'Участок': 'Plot',
    'Torpaq': 'Land',
    'Qaraj': 'Garage',
    'Гараж': 'Garage'
}

frames['building_type_unified'] = frames['building_type'].replace(building_type_mapping)

frames['is_near_metro'] = (frames['location'].str.contains('m\.', case=False) |
                               frames['address_all'].str.contains('m\.', case=False) |
                               frames['description'].str.contains('m\.', case=False) |
                               frames['location'].str.contains('metro', case=False) |
                               frames['address_all'].str.contains('metro', case=False) |
                               frames['description'].str.contains('metro', case=False)).astype(int)

features = ['view_count','seller_type_encoded','building_type_unified','is_near_metro']
target = ['price']
features=frames[features].reset_index( drop=True)
target=frames[target]

dummy_building_type = pd.get_dummies(features['building_type_unified'], prefix='building_type')
features = pd.concat([features, dummy_building_type], axis=1)
for col in features.columns:
    if features[col].dtype == bool:
        features[col] = features[col].astype(int)
features.drop('building_type_unified', axis=1, inplace=True)

In [10]:
import pandas as pd

def load_and_concat_data(file1, file2):
    data_1 = pd.read_csv(file1)
    data_2 = pd.read_csv(file2)
    return pd.concat([data_1, data_2], ignore_index=True).drop_duplicates(keep='last').dropna().reset_index(drop=True)

def extract_view_count(df):
    df['view_count'] = df['view'].str.extract(r'(?i)(?:Baxışların sayı|Просмотров): (\d+)')
    return df

def encode_seller_type(df):
    seller_type_mapping = {
        'vasitəçi (agent)': 0,
        'посредник (агент)': 0,
        'собственник': 1,
        'mülkiyyətçi': 1
    }
    df['seller_type_encoded'] = df['seller_type'].replace(seller_type_mapping)
    df.drop('seller_type', axis=1, inplace=True)
    return df

def clean_and_sort_data(df):
    df['price'] = df['price'].str.replace(' ', '').astype(float)
    return df.sort_values(by='price', ascending=True)

def map_building_type(df):
    building_type_mapping = {
        'Köhnə tikili': 'Old building',
        'Yeni tikili': 'New building',
        # ... Add more mappings here ...
    }
    df['building_type_unified'] = df['building_type'].replace(building_type_mapping)
    return df

def check_near_metro(df):
    df['is_near_metro'] = (df['location'].str.contains('m\.', case=False) |
                           df['address_all'].str.contains('m\.', case=False) |
                           df['description'].str.contains('m\.', case=False) |
                           df['location'].str.contains('metro', case=False) |
                           df['address_all'].str.contains('metro', case=False) |
                           df['description'].str.contains('metro', case=False)).astype(int)
    return df

def process_data(df):
    df = extract_view_count(df)
    df = encode_seller_type(df)
    df = clean_and_sort_data(df)
    df = map_building_type(df)
    df = check_near_metro(df)
    return df

if __name__ == '__main__':
    file1 = 'bina_az_19092023.csv'
    file2 = 'bina_az_21092023.csv'
    
    frames = load_and_concat_data(file1, file2)
    frames = process_data(frames)

    features = ['view_count', 'seller_type_encoded', 'building_type_unified', 'is_near_metro']
    target = ['price']

    features = frames[features].reset_index(drop=True)
    target = frames[target]

    # One-hot encode the 'building_type_unified' column
    dummy_building_type = pd.get_dummies(features['building_type_unified'], prefix='building_type')
    features = pd.concat([features, dummy_building_type], axis=1)

    # Drop the original 'building_type_unified' column
    features.drop('building_type_unified', axis=1, inplace=True)
    # Make sure your target variable has the same number of samples as features


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust n_estimators as needed

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared (R2) Score: {r2}")



C:\Users\Ismat\.conda\envs\venv_scrapy\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 237404058000.58228
R-squared (R2) Score: -0.10886799544308112
